# Administrative Descriptive Stats Preparation

After appending travel time information to each populated place in an administrative center we can prepare any number of descriptive stats. Given the quantity of data in question these are best prepared with Dask Dataframes. This notebook separates out the descriptive stats preparations,

In [1]:
import dask
import coiled
from dask.distributed import Client, LocalCluster, Lock
from dask.utils import SerializableLock
import dask.dataframe as dd

import pandas as pd
import geopandas as gpd
import spatialpandas as sp
import dask_geopandas as dg

import rioxarray as rx
import xarray as xr

import re
import os

from dask_control import *
from raster_ex import *

from spatialpandas.geometry import (
    PointArray, MultiPointArray, LineArray,
    MultiLineArray, PolygonArray, MultiPolygonArray
)

import numpy as np
from datetime import date

## Setup

**Files and variables**

Today's date

In [2]:
today = date.today().strftime("%d%m%y")

Directories

In [3]:
geo_dir = r'P:\PAK\GEO'
data_dir = r'../../data'

dest_dir = r'destinations'
acc_dir = r'access'
tab_dir = r'tabular'

In [4]:
POINTS_URL = r'tabular/df_pixels_final-*.csv'

Projections

In [5]:
# change this to whatever the desired output projection is
DEST_CRS = 'EPSG:32642'

dcrs_int = int(re.findall('[0-9]+',DEST_CRS)[0])
dcrs_int

32642

**Initiate Dask Client**

In [6]:
client=get_dask_client(cluster_type='local',n_workers=4,processes=True,threads_per_worker=8)
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 4
Total threads: 32,Total memory: 63.98 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:8786,Workers: 4
Dashboard: http://127.0.0.1:8787/status,Total threads: 32
Started: Just now,Total memory: 63.98 GiB
Comm: tcp://10.175.66.81:58327,Total threads: 8
Dashboard: http://10.175.66.81:58328/status,Memory: 16.00 GiB
Nanny: tcp://127.0.0.1:58264,


## Loading data

Pixel level data

In [7]:
# df_pixels = dd.read_parquet(os.path.join(data_dir,acc_dir,'tabular/df_pixels_final.parquet'),
#                                     na_values = ' ',
#                                     blocksize=100e6)\
#                                     .sort_values('Adm2_Code')\
#                                     .set_index('ix', drop=True, sorted=False)

In [8]:
df_pixels = dd.read_csv(os.path.join(data_dir,acc_dir,POINTS_URL),header=0,
                                    na_values = ' ',
                                    blocksize=100e6)\
                                    .sort_values('Adm2_Code')\
                                    .set_index('ix', drop=True, sorted=False)

In [9]:
df_pixels

,POP,x_4326,y_4326,x,y,ADM1_EN,ADM2_EN,ADM3_EN,Adm1_Code,Adm2_Code,Adm3_Code,childwalk_dry_District_HQs,childwalk_dry_education_allboys,childwalk_dry_education_allgirls,childwalk_dry_education_boys,childwalk_dry_education_boys_high,childwalk_dry_education_boys_middle,childwalk_dry_education_boys_primary,childwalk_dry_education_girls,childwalk_dry_education_girls_high,childwalk_dry_education_girls_middle,childwalk_dry_education_girls_primary,childwalk_dry_education_high,childwalk_dry_education_middle,childwalk_dry_education_primary,childwalk_dry_health_family,childwalk_dry_health_pharmacies,childwalk_dry_health_primary,childwalk_dry_health_private,childwalk_dry_health_public,childwalk_dry_health_secondary,childwalk_dry_health_tertiary,childwalk_dry_markets_All,childwalk_dry_markets_Central,childwalk_dry_Provincial_HQ,childwalk_msn_District_HQs,childwalk_msn_education_allboys,childwalk_msn_education_allgirls,childwalk_msn_education_boys,childwalk_msn_education_boys_high,childwalk_msn_education_boys_middle,childwalk_msn_education_boys_primary,childwalk_msn_education_girls,childwalk_msn_education_girls_high,childwalk_msn_education_girls_middle,childwalk_msn_education_girls_primary,childwalk_msn_education_high,childwalk_msn_education_middle,childwalk_msn_education_primary,childwalk_msn_health_family,childwalk_msn_health_pharmacies,childwalk_msn_health_primary,childwalk_msn_health_private,childwalk_msn_health_public,childwalk_msn_health_secondary,childwalk_msn_health_tertiary,childwalk_msn_markets_All,childwalk_msn_markets_Central,childwalk_msn_Provincial_HQ,childwalk_winter_District_HQs,childwalk_winter_education_allboys,childwalk_winter_education_allgirls,childwalk_winter_education_boys,childwalk_winter_education_boys_high,childwalk_winter_education_boys_middle,childwalk_winter_education_boys_primary,childwalk_winter_education_girls,childwalk_winter_education_girls_high,childwalk_winter_education_girls_middle,childwalk_winter_education_girls_primary,childwalk_winter_education_high,childwalk_winter_education_middle,childwalk_winter_education_primary,childwalk_winter_health_family,childwalk_winter_health_pharmacies,childwalk_winter_health_primary,childwalk_winter_health_private,childwalk_winter_health_public,childwalk_winter_health_secondary,childwalk_winter_health_tertiary,childwalk_winter_markets_All,childwalk_winter_markets_Central,childwalk_winter_Provincial_HQ,dry_District_HQs,dry_education_allboys,dry_education_allgirls,dry_education_boys,dry_education_boys_high,dry_education_boys_middle,dry_education_boys_primary,dry_education_girls,dry_education_girls_high,dry_education_girls_middle,dry_education_girls_primary,dry_education_high,dry_education_middle,dry_education_primary,dry_health_family,dry_health_pharmacies,dry_health_primary,dry_health_private,dry_health_public,dry_health_secondary,dry_health_tertiary,dry_markets_All,dry_markets_Central,dry_Provincial_HQ,msn_District_HQs,msn_education_allboys,msn_education_allgirls,msn_education_boys,msn_education_boys_high,msn_education_boys_middle,msn_education_boys_primary,msn_education_girls,msn_education_girls_high,msn_education_girls_middle,msn_education_girls_primary,msn_education_high,msn_education_middle,msn_education_primary,msn_health_family,msn_health_pharmacies,msn_health_primary,msn_health_private,msn_health_public,msn_health_secondary,msn_health_tertiary,msn_markets_All,msn_markets_Central,msn_Provincial_HQ,walk_dry_District_HQs,walk_dry_education_allboys,walk_dry_education_allgirls,walk_dry_education_boys,walk_dry_education_boys_high,walk_dry_education_boys_middle,walk_dry_education_boys_primary,walk_dry_education_girls,walk_dry_education_girls_high,walk_dry_education_girls_middle,walk_dry_education_girls_primary,walk_dry_education_high,walk_dry_education_middle,walk_dry_education_primary,walk_dry_health_family,walk_dry_health_pharmacies,walk_dry_health_primary,walk_dry_health_private,walk_dry_health_public,walk_dry_health_secondary,walk_dry_health_tertiary,walk_dry_marke

In [10]:
# persist as well will be conducting many operations on this dataset and want to branch off from it
df_pixels = df_pixels.persist()

Reference rasters

In [11]:
rasters = {}
rlimit = 9999
r_ct = 0
for file in os.listdir(os.path.join(data_dir,acc_dir,'current/master')):
    #if file.endswith("_COG.tif"):
    if file.startswith("Current_"):
        acc_rast = re.search(r'Current_(.*?).tif',os.path.basename(file)).group(1)
        rasters[acc_rast] = f"{data_dir}/{acc_dir}/current/master/{file}"
        r_ct = r_ct + 1
        if r_ct >= rlimit:
            break

In [12]:
# rasters

Use reference rasters to ID columns to extract

In [13]:
access_cols = [acc_col for acc_col in rasters]

In [14]:
access_cols[::10]

['childwalk_dry_District_HQs',
 'childwalk_dry_education_girls_primary',
 'childwalk_dry_health_tertiary',
 'childwalk_msn_education_boys_primary',
 'childwalk_msn_health_primary',
 'childwalk_winter_education_allgirls',
 'childwalk_winter_education_middle',
 'childwalk_winter_markets_Central',
 'dry_education_girls_high',
 'dry_health_public',
 'msn_education_boys_high',
 'msn_health_family',
 'walk_dry_District_HQs',
 'walk_dry_education_girls_primary',
 'walk_dry_health_tertiary',
 'walk_msn_education_boys_primary',
 'walk_msn_health_primary',
 'walk_winter_education_allgirls',
 'walk_winter_education_middle',
 'walk_winter_markets_Central',
 'winter_education_girls_high',
 'winter_health_public']

## Aggregation code

### Binning by travel time

Filter down data

In [15]:
# Make a dataframe with just the columns we'll be using
# Need to change range numbers if we add columns

dft = dd.multi.concat([df_pixels[["POP","ADM2_EN","ADM3_EN","Adm2_Code","Adm3_Code"]],df_pixels[access_cols]],axis=1)

In [16]:
dft = dft.compute()

Travel time ranges

In [17]:
tt_bins = [0, 0.5, 1, 2, 4, 8, 16, 10000]
tt_bin_labels = ["0 - 30 minutes", "31 - 60 minutes", "1 - 2 hours", "2 - 4 hours", "4 - 8 hours", "8 - 16 hours", "16+ hours"]

# rename dict
tt_rename_dct = {
    1 : "0 - 30 minutes",
    2 : "31 - 60 minutes",
    3 : "1 - 2 hours",
    4 : "2 - 4 hours",
    5 : "4 - 8 hours",
    6 : "8 - 16 hours",
    7 : "16+ hours"}

In [18]:
# # Alternately calculate using PLSM TT breaks for direct comparison with survey data

# tt_bins = [0, 0.249999, 0.49999, 0.749999, 0.999999, 10000]
# tt_bin_labels = ["0 - 14 minutes", "15 - 29 minutes", "30 - 44 minutes", "45 - 59 hours", "60+ minutes"]

# # rename dict
# tt_rename_dct = {
#     1 : "0 - 14 minutes",
#     2 : "15 - 29 minutes",
#     3 : "30 - 44 minutes",
#     4 : "45 - 59 hours",
#     5 :  "60+ minutes"}

Functions

In [19]:
def long_per_indicator(df,indicator,adm_col):
    
    indic_label = indicator.replace('_COG','')
    
    # pivot the data for just that indicator, with the column VALUES = the population value for that pixel
    pop_total = df.pivot_table(index = adm_col, columns=indicator, 
                 values = 'POP', aggfunc = 'sum', fill_value = 0)
    
    # divide by the rowsum to get the % of population falling in each travel category, per admin area
    pop_pct = pop_total.div(np.nansum(pop_total,axis=1),axis=0)
    
    # create labels
    pop_total['indicator'] = indic_label
    pop_pct['indicator'] = indic_label
    
    # remove the multi-index, compress in long format with the adm and indicator as labels, and then change labels/sort
    pop_pct = pop_pct.reset_index()\
                            .melt(id_vars=[adm_col,'indicator'])\
                            .rename({indicator:'travel_time_range','value':'pop_pct'},axis=1)
    
    pop_total = pop_total.reset_index()\
                            .melt(id_vars=[adm_col,'indicator'])\
                            .rename({indicator:'travel_time_range','value':'pop_total'},axis=1)
    
    long_indic = pd.concat([pop_pct,pop_total[['pop_total']]],axis=1,ignore_index=False)
    
    return long_indic

Replace values with bin number (via numpy), then rename bin numbers using dict created above

In [20]:
%timeit
dft[access_cols] = pd.DataFrame(np.digitize(dft[access_cols], bins=tt_bins),columns=dft[access_cols].columns, index=dft.index)

In [21]:
%timeit
dft[access_cols] = dft[access_cols].apply(lambda x: x.replace(tt_rename_dct))

In [22]:
dft

,POP,ADM2_EN,ADM3_EN,Adm2_Code,Adm3_Code,childwalk_dry_District_HQs,childwalk_dry_education_allboys,childwalk_dry_education_allgirls,childwalk_dry_education_boys,childwalk_dry_education_boys_high,...,winter_health_family,winter_health_pharmacies,winter_health_primary,winter_health_private,winter_health_public,winter_health_secondary,winter_health_tertiary,winter_markets_All,winter_markets_Central,winter_Provincial_HQ
ix,,,,,,,,,,,,,,,,,,,,,
2148470,5.587176,Chitral,Mastuj,PK206,PK20602,16+ hours,8 - 16 hours,16+ hours,8 - 16 hours,16+ hours,...,16+ hours,16+ hours,16+ hours,16+ hours,16+ hours,16+ hours,16+ hours,16+ hours,16+ hours,16+ hours
2148497,5.587176,Chitral,Mastuj,PK206,PK20602,16+ hours,16+ hours,16+ hours,16+ hours,16+ hours,...,16+ hours,16+ hours,16+ hours,16+ hours,16+ hours,16+ hours,16+ hours,16+ hours,16+ hours,16+ hours
2148611,5.587176,Chitral,Mastuj,PK206,PK20602,16+ hours,4 - 8 hours,16+ hours,4 - 8 hours,16+ hours,...,8 - 16 hours,4 - 8 hours,8 - 16 hours,8 - 16 hours,4 - 8 hours,8 - 16 hours,16+ hours,8 - 16 hours,16+ hours,16+ hours
2148619,5.587176,Chitral,Mastuj,PK206,PK20602,16+ hours,4 - 8 hours,16+ hours,4 - 8 hours,16+ hours,...,8 - 16 hours,4 - 8 hours,8 - 16 hours,8 - 16 hours,4 - 8 hours,8 - 16 hours,16+ hours,8 - 16 hours,16+ hours,16+ hours
2148666,5.587176,Chitral,Mastuj,PK206,PK20602,16+ hours,4 - 8 hours,16+ hours,4 - 8 hours,16+ hours,...,8 - 16 hours,4 - 8 hours,8 - 16 hours,8 - 16 hours,4 - 8 hours,8 - 16 hours,16+ hours,8 - 16 hours,16+ hours,16+ hours
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6883657,14.151661,FR Dera Ismail Khan,Drazanda,PK240,PK24001,16+ hours,4 - 8 hours,4 - 8 hours,4 - 8 hours,4 - 8 hours,...,4 - 8 hours,4 - 8 hours,4 - 8 hours,4 - 8 hours,4 - 8 hours,4 - 8 hours,4 - 8 hours,4 - 8 hours,4 - 8 hours,8 - 16 hours
6887031,14.151661,FR Dera Ismail Khan,Drazanda,PK240,PK24001,16+ hours,4 - 8 hours,4 - 8 hours,4 - 8 hours,4 - 8 hours,...,4 - 8 hours,4 - 8 hours,4 - 8 hours,4 - 8 hours,4 - 8 hours,4 - 8 hours,4 - 8 hours,4 - 8 hours,4 - 8 hours,8 - 16 hours
6888123,14.151661,FR Dera Ismail Khan,Drazanda,PK240,PK24001,16+ hours,4 - 8 hours,4 - 8 hours,4 - 8 hours,4 - 8 hours,...,4 - 8 hours,4 - 8 hours,4 - 8 hours,4 - 8 hours,4 - 8 hours,4 - 8 hours,4 - 8 hours,4 - 8 hours,4 - 8 hours,8 - 16 hours


For each indicator, pivot data by administrative unit, calculate the pct of total population per travel time bin, and reshape the data into a long format.</br>Then merge all these reshaped long tables into one master table 

In [23]:
long_data_lst_adm2 = []
long_data_lst_adm3 = []

for i in access_cols:
    
    long_i_adm2 = long_per_indicator(dft,i,'Adm2_Code')
    long_i_adm3 = long_per_indicator(dft,i,'Adm3_Code')
    
    long_data_lst_adm2.append(long_i_adm2)
    long_data_lst_adm3.append(long_i_adm3)

In [24]:
tt_bin_labels

['0 - 30 minutes',
 '31 - 60 minutes',
 '1 - 2 hours',
 '2 - 4 hours',
 '4 - 8 hours',
 '8 - 16 hours',
 '16+ hours']

In [25]:
# concatenate
long_acc_indicators_adm2 = pd.concat(long_data_lst_adm2,ignore_index=True)
long_acc_indicators_adm3 = pd.concat(long_data_lst_adm3,ignore_index=True)

# convert tt ranges to categorical and order appropriately
long_acc_indicators_adm2['travel_time_range'] = long_acc_indicators_adm2.travel_time_range.astype('category').cat.set_categories(tt_bin_labels)
long_acc_indicators_adm3['travel_time_range'] = long_acc_indicators_adm3.travel_time_range.astype('category').cat.set_categories(tt_bin_labels)

# order as desired
long_acc_indicators_adm2 = long_acc_indicators_adm2.sort_values(['Adm2_Code','indicator','travel_time_range']).reset_index(drop=True)
long_acc_indicators_adm3 = long_acc_indicators_adm3.sort_values(['Adm3_Code','indicator','travel_time_range']).reset_index(drop=True)

In [26]:
long_acc_indicators_adm2

,Adm2_Code,indicator,travel_time_range,pop_pct,pop_total
0,PK201,childwalk_dry_District_HQs,0 - 30 minutes,0.017324,12652.987782
1,PK201,childwalk_dry_District_HQs,31 - 60 minutes,0.028400,20742.602921
2,PK201,childwalk_dry_District_HQs,1 - 2 hours,0.083855,61245.890186
3,PK201,childwalk_dry_District_HQs,2 - 4 hours,0.247428,180716.630268
4,PK201,childwalk_dry_District_HQs,4 - 8 hours,0.355357,259546.290173
...,...,...,...,...,...
57451,PK244,winter_markets_Central,1 - 2 hours,0.324304,6296.068492
57452,PK244,winter_markets_Central,2 - 4 hours,0.554705,10769.084268
57453,PK244,winter_markets_Central,4 - 8 hours,0.120216,2333.880911
57454,PK244,winter_markets_Central,8 - 16 hours,0.000775,15.044154


Calculate cumulative sums per indicator and the Adm2_Code for Adm3 datasets

In [27]:
long_acc_indicators_adm2['pop_pct_csum'] = long_acc_indicators_adm2.groupby(['Adm2_Code','indicator'])['pop_pct'].cumsum(axis=0)
long_acc_indicators_adm3['pop_pct_csum'] = long_acc_indicators_adm3.groupby(['Adm3_Code','indicator'])['pop_pct'].cumsum(axis=0)

long_acc_indicators_adm2['pop_total_csum'] = long_acc_indicators_adm2.groupby(['Adm2_Code','indicator'])['pop_total'].cumsum(axis=0)
long_acc_indicators_adm3['pop_total_csum'] = long_acc_indicators_adm3.groupby(['Adm3_Code','indicator'])['pop_total'].cumsum(axis=0)


In [28]:
long_acc_indicators_adm3['Adm2_Code'] = long_acc_indicators_adm3['Adm3_Code'].str[:5]

In [29]:
long_acc_indicators_adm2

,Adm2_Code,indicator,travel_time_range,pop_pct,pop_total,pop_pct_csum,pop_total_csum
0,PK201,childwalk_dry_District_HQs,0 - 30 minutes,0.017324,12652.987782,0.017324,12652.987782
1,PK201,childwalk_dry_District_HQs,31 - 60 minutes,0.028400,20742.602921,0.045723,33395.590702
2,PK201,childwalk_dry_District_HQs,1 - 2 hours,0.083855,61245.890186,0.129578,94641.480888
3,PK201,childwalk_dry_District_HQs,2 - 4 hours,0.247428,180716.630268,0.377006,275358.111156
4,PK201,childwalk_dry_District_HQs,4 - 8 hours,0.355357,259546.290173,0.732363,534904.401329
...,...,...,...,...,...,...,...
57451,PK244,winter_markets_Central,1 - 2 hours,0.324304,6296.068492,0.324304,6296.068492
57452,PK244,winter_markets_Central,2 - 4 hours,0.554705,10769.084268,0.879009,17065.152760
57453,PK244,winter_markets_Central,4 - 8 hours,0.120216,2333.880911,0.999225,19399.033671
57454,PK244,winter_markets_Central,8 - 16 hours,0.000775,15.044154,1.000000,19414.077825


In [30]:
long_acc_indicators_adm3

,Adm3_Code,indicator,travel_time_range,pop_pct,pop_total,pop_pct_csum,pop_total_csum,Adm2_Code
0,PK20101,childwalk_dry_District_HQs,0 - 30 minutes,0.017324,12652.987782,0.017324,12652.987782,PK201
1,PK20101,childwalk_dry_District_HQs,31 - 60 minutes,0.028400,20742.602921,0.045723,33395.590702,PK201
2,PK20101,childwalk_dry_District_HQs,1 - 2 hours,0.083855,61245.890186,0.129578,94641.480888,PK201
3,PK20101,childwalk_dry_District_HQs,2 - 4 hours,0.247428,180716.630268,0.377006,275358.111156,PK201
4,PK20101,childwalk_dry_District_HQs,4 - 8 hours,0.355357,259546.290173,0.732363,534904.401329,PK201
...,...,...,...,...,...,...,...,...
175387,PK24401,winter_markets_Central,1 - 2 hours,0.324304,6296.068492,0.324304,6296.068492,PK244
175388,PK24401,winter_markets_Central,2 - 4 hours,0.554705,10769.084268,0.879009,17065.152760,PK244
175389,PK24401,winter_markets_Central,4 - 8 hours,0.120216,2333.880911,0.999225,19399.033671,PK244
175390,PK24401,winter_markets_Central,8 - 16 hours,0.000775,15.044154,1.000000,19414.077825,PK244


Export final long data

In [31]:
long_acc_indicators_adm2.to_csv(os.path.join(data_dir,tab_dir,f'final//adm2_acc_indicators_long_{today}.csv'))
long_acc_indicators_adm3.to_csv(os.path.join(data_dir,tab_dir,f'final//adm3_acc_indicators_long_{today}.csv'))

## Compute standard deviation access values, weighted by pixel population

In [31]:
# Make a dataframe with just the columns we'll be using
# Need to change range numbers if we add columns

dfsd = dd.multi.concat([df_pixels[["POP","ADM2_EN","ADM3_EN","Adm2_Code","Adm3_Code","adm2_pop","adm3_pop","wt_adm_2","wt_adm_3"]],df_pixels[access_cols]],axis=1)

In [32]:
dfsd = dfsd.compute()

In [33]:
# # Get Pops by Adm2_Code
# adm2_pop = dfsd.groupby('Adm2_Code')['POP'].sum().to_frame("adm2_pop")

# # Get Pops by CBS Ward
# adm3_pop = dfsd.groupby('Adm3_Code')['POP'].sum().to_frame("adm3_pop")

In [34]:
# # Merge the Pops into Ref DF
# df_pixels = dd.merge(df_pixels, adm2_pop, how = 'left', left_on="Adm2_Code", right_index=True)
# df_pixels = dd.merge(df_pixels, adm3_pop, how = 'left', left_on="Adm3_Code", right_index=True)

# #points = points.persist()
# df_pixels = df_pixels.persist()

In [35]:
# # Calculate the population weight of each pixel within its enclosing admin area -- e.g. 10 pixel population for a 100 population admin - 0.1 weight
# df_pixels['wt_adm_2'] = (df_pixels['POP'] / df_pixels['adm2_pop'])
# df_pixels['wt_adm_3'] = (df_pixels['POP'] / df_pixels['adm3_pop'])

In [36]:
adm2_avg_cols = []
adm3_avg_cols = []

for rkey in rasters:
    hrs_col = f"{rkey}"
    avg_col_adm_2 = f"{rkey}_std_adm2"
    avg_col_adm_3 = f"{rkey}_std_adm3"
    dfsd[avg_col_adm_2] = dfsd[hrs_col] * dfsd['wt_adm_2']
    dfsd[avg_col_adm_3] = dfsd[hrs_col] * dfsd['wt_adm_3']
    
    adm2_avg_cols.append(avg_col_adm_2)
    adm3_avg_cols.append(avg_col_adm_3)

C:\Users\wb467985\AppData\Local\Temp/ipykernel_4344/2582803638.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  dfsd[avg_col_adm_2] = dfsd[hrs_col] * dfsd['wt_adm_2']
C:\Users\wb467985\AppData\Local\Temp/ipykernel_4344/2582803638.py:9: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  dfsd[avg_col_adm_3] = dfsd[hrs_col] * dfsd['wt_adm_3']


In [37]:
print(",".join(adm2_avg_cols))
print(",".join(adm3_avg_cols))

childwalk_dry_District_HQs_COG_std_adm2,childwalk_dry_education_allboys_COG_std_adm2,childwalk_dry_education_allgirls_COG_std_adm2,childwalk_dry_education_boys_COG_std_adm2,childwalk_dry_education_boys_high_COG_std_adm2,childwalk_dry_education_boys_middle_COG_std_adm2,childwalk_dry_education_boys_primary_COG_std_adm2,childwalk_dry_education_girls_COG_std_adm2,childwalk_dry_education_girls_high_COG_std_adm2,childwalk_dry_education_girls_middle_COG_std_adm2,childwalk_dry_education_girls_primary_COG_std_adm2,childwalk_dry_education_high_COG_std_adm2,childwalk_dry_education_middle_COG_std_adm2,childwalk_dry_education_primary_COG_std_adm2,childwalk_dry_health_family_COG_std_adm2,childwalk_dry_health_pharmacies_COG_std_adm2,childwalk_dry_health_primary_COG_std_adm2,childwalk_dry_health_private_COG_std_adm2,childwalk_dry_health_public_COG_std_adm2,childwalk_dry_health_secondary_COG_std_adm2,childwalk_dry_health_tertiary_COG_std_adm2,childwalk_dry_markets_All_COG_std_adm2,childwalk_dry_markets

In [38]:
adm2_sd_final = dfsd.groupby(['Adm2_Code'])[adm2_avg_cols].std().reset_index()
adm3_sd_final = dfsd.groupby(['Adm3_Code'])[adm3_avg_cols].std().reset_index()

#### Export

Export names

In [39]:
adm2_sd_output = "../../data/tabular/processed/adm2_sd_211015.csv"
adm3_sd_output = "../../data/tabular/processed/adm3_sd_211015.csv"

Actually export

In [40]:
%time
adm2_sd_final.to_csv(adm2_sd_output)
adm3_sd_final.to_csv(adm3_sd_output)

Wall time: 997 µs


### Compute mean access values, weighted by pixel population

In [ ]:
# Get Pops by Adm2_Code
adm2_pop = df_pixels.groupby('Adm2_Code')['POP'].sum().to_frame("adm2_pop")

# Get Pops by CBS Ward
adm3_pop = df_pixels.groupby('Adm3_Code')['POP'].sum().to_frame("adm3_pop")

In [ ]:
# Merge the Pops into Ref DF
df_pixels = dd.merge(df_pixels, adm2_pop, how = 'left', left_on="Adm2_Code", right_index=True)
df_pixels = dd.merge(df_pixels, adm3_pop, how = 'left', left_on="Adm3_Code", right_index=True)

#points = points.persist()
df_pixels = df_pixels.persist()

In [ ]:
# Calculate the population weight of each pixel within its enclosing admin area -- e.g. 10 pixel population for a 100 population admin - 0.1 weight
df_pixels['wt_adm_2'] = (df_pixels['POP'] / df_pixels['adm2_pop'])
df_pixels['wt_adm_3'] = (df_pixels['POP'] / df_pixels['adm3_pop'])

In [ ]:
adm2_avg_cols = []
adm3_avg_cols = []

for rkey in rasters:
    hrs_col = f"{rkey}"
    avg_col_adm_2 = f"{rkey}_avg_adm2"
    avg_col_adm_3 = f"{rkey}_avg_adm3"
    df_pixels[avg_col_adm_2] = df_pixels[hrs_col] * df_pixels['wt_adm_2']
    df_pixels[avg_col_adm_3] = df_pixels[hrs_col] * df_pixels['wt_adm_3']
    
    adm2_avg_cols.append(avg_col_adm_2)
    adm3_avg_cols.append(avg_col_adm_3)

In [ ]:
print(",".join(adm2_avg_cols))
print(",".join(adm3_avg_cols))

In [ ]:
adm2_final = df_pixels.groupby(['Adm2_Code'])[adm2_avg_cols].sum().reset_index()
adm3_final = df_pixels.groupby(['Adm3_Code'])[adm3_avg_cols].sum().reset_index()

#### Export

Export names

In [ ]:
adm2_output = "../../data/outputs/adm2_final.csv"
adm3_output = "../../data/outputs/adm3_final.csv"
df_pixels_out = "../../data/outputs/df_pixels_final.csv"

Actually export

In [ ]:
%time
adm2_final.to_csv(adm2_output, single_file=True)
adm3_final.to_csv(adm3_output, single_file=True)

In [ ]:
client.close()